In [1]:
### Import
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from scipy.signal import find_peaks_cwt
import time

%matplotlib inline

In [2]:
### Load Camera Calibration Pickle Data
dist_data = pickle.load( open( "dist_pickle.p", "rb" ) )
camera_mtx = dist_data["mtx"]
camera_dist = dist_data["dist"]

### Load Images
images = glob.glob('test_images/*.jpg')

In [3]:
# Display Function
def disp_img(original_image, augmented_image, new_cmap=None, aug_title = ""):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(original_image)
    ax2.imshow(augmented_image, cmap=new_cmap)
    ax1.set_title('Original Image', fontsize=30)
    ax2.set_title('Augmented Image: ' + aug_title, fontsize=30)
    plt.show()

In [4]:
### Color Thresholding
def threshold(image, thresh_val):
    return cv2.threshold(image, thresh_val[0], thresh_val[1], cv2.THRESH_BINARY)

def color_threshold(img):
    
    # Channel based thresholding
    # HLS
    undist_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    undist_S = undist_hls[:,:,2]
    thresh_S = (180, 255)

    binary_hls_S = np.zeros_like(undist_S)
    _, binary_hls_S = threshold(undist_S, thresh_S)
    

    # LUV
    undist_luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    undist_L = undist_luv[:,:,0]
    thresh_L = (225, 255)

    binary_luv_L = np.zeros_like(undist_L)
    _, binary_luv_L = threshold(undist_L, thresh_L)
    

    # LAB
    undist_lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    undist_B = undist_lab[:,:,2]
    thresh_B = (150, 255)
    binary_lab_B = np.zeros_like(undist_B)
    _, binary_lab_B = threshold(undist_B, thresh_B)
    
    
    # Combined Color Thresholding
    combined_color_binary = np.zeros_like(binary_lab_B)
    
    #combined_color_binary[(binary_hls_S == 255) | (binary_luv_L == 255) | (binary_lab_B == 255)] = 255
    combined_color_binary[(binary_luv_L == 255) | (binary_lab_B == 255)] = 255
    return combined_color_binary

In [5]:
### Perspective Transform

def undistort_img(image):
    # Undistort test image
    undist_image = cv2.undistort(image, camera_mtx, camera_dist, None, camera_mtx)

    img_size = (undist_image.shape[1], undist_image.shape[0])
    
    # Lane line vertices
    # Upper and low are based on visual locations, not grid locations
    center_x = img_size[0]//2
    upper_y = img_size[1]//1.5
    low_y = img_size[1]
    upper_left_x = center_x//1.33
    upper_right_x = center_x//0.80
    low_left_x = 0
    low_right_x = 2*center_x
    
    # Calculate source points based on fractions of imade dimensions
    src_corners = np.float32([[low_left_x, low_y], 
                              [upper_left_x, upper_y], 
                              [upper_right_x, upper_y],
                              [low_right_x, low_y]])
   
    
    # Calculate destination points based on entire image's dimensions.
    dst_corners = np.float32([[0, img_size[1]],
                              [0, 0],
                              [img_size[0],0],
                              [img_size[0], img_size[1]]])
    
    return undist_image, src_corners, dst_corners

def perspective_transform(image):
    # Calculate perspective transform
    
    undist_image, src_corners, dst_corners = undistort_img(image)    
    
    img_size = (undist_image.shape[1], undist_image.shape[0])
    
    M = cv2.getPerspectiveTransform(src_corners, dst_corners)

    warped = cv2.warpPerspective(undist_image, M, img_size)
    
    M_inv = cv2.getPerspectiveTransform(dst_corners, src_corners)

    # Draw points and lines to mark region for transform
    for i in range(4):
        cv2.circle(undist_image, (src_corners[i,0], src_corners[i,1]), 6, (255, 0, 0), 6)
    for i in range(4):
        cv2.line(undist_image, 
                 (src_corners[i-1,0], src_corners[i-1,1]), 
                 (src_corners[i,0], src_corners[i,1]),  
                 (0,255,0), 2)
    
        
    return warped, M_inv

In [6]:
### Color Thresholding on Bird-Eye View

def thresholded_img(image):
    # Image thresholding
    warped, M_inv = perspective_transform(image)
    blurred_warped = cv2.GaussianBlur(warped,(5,5),0)

    # Color Thresholding
    combined_color_binary = color_threshold(blurred_warped)
    
    return combined_color_binary, M_inv

In [7]:
### Lane Finding
def lane_coords(image):
    
    # Define image height and window size
    img_dim_y = image.shape[0]
    img_dim_x = image.shape[1]
    img_slices = 10
    win_size = img_dim_y//img_slices
    
    # Histogram
    indexes = []
    hist_val = []
    
    for i in range(img_slices):
        histogram = np.mean(image[(img_slices-i-1)*win_size:(img_slices-i)*win_size,:], axis=0)
        
        # Histogram search area
        left_half_min = 0
        left_half_max = img_dim_x//2.5
        right_half = img_dim_x-(img_dim_x//2.2)
        
        # Histogram peaks
        hist_peaksl = np.argmax(histogram[left_half_min:left_half_max])
        hist_peaksr = np.argmax(histogram[right_half:])
        
        # Reduce noise
        if histogram[hist_peaksl+left_half_min] > 0.01:
            hist_peaksl = int(hist_peaksl+left_half_min)
        else:
            hist_peaksl = 0
        
        if histogram[hist_peaksr + right_half] > 0.01:
            hist_peaksr = int(hist_peaksr + right_half)
        else:
            hist_peaksr = 0
        
        # Append indices
        indexes.append([hist_peaksl, hist_peaksr])
    
    
    left_lane_idx = []
    right_lane_idx = []
    
    
    for i in range(img_slices):
        # Define window y positions
        win_y1 = (img_slices-i)*win_size
        win_y2 = (img_slices-i)*win_size - win_size        

        # Draw boxes and get indices of thresholded lane points
        # Left Lane
        if (indexes[i][0] != 0):
            # Define window x positions
            win_x1l = indexes[i][0] - (win_size//2)
            win_x2l = indexes[i][0] + (win_size//2)
            
            # Identify lane points where line was detected
            left_lane_idx_local = np.argwhere(image[win_y2:win_y1, win_x1l:win_x2l] > 0)
            # Append to list of lane indices and apply frame of reference transformation
            left_lane_idx.append(left_lane_idx_local + [win_y2,win_x1l])

        # Right lane
        if(indexes[i][1] != 0):
            # Define window x positions
            win_x1r = indexes[i][1] - (win_size//2)
            win_x2r = indexes[i][1] + (win_size//2)
            
            # Identify lane points where line was detected
            right_lane_idx_local = np.argwhere(image[win_y2:win_y1, win_x1r:win_x2r] > 0)
            
            # Append to list of lane indices and apply frame of reference transformation
            right_lane_idx.append(right_lane_idx_local + [win_y2,win_x1r])            
    
    # Concatenate all lane points to respective lane variables
    left_lane_idx = np.concatenate(left_lane_idx[:], axis=0)
    right_lane_idx = np.concatenate(right_lane_idx[:], axis=0)

    
    return left_lane_idx, right_lane_idx

In [8]:
### Draw Lane Points on new image
def draw_lane_points(img, left_lane_idx, right_lane_idx):
    
    new_img = np.zeros_like(img)
    for i in range(len(left_lane_idx)):
        for j in range(len(left_lane_idx[i])):
            cv2.circle(new_img, (left_lane_idx[i][1],left_lane_idx[i][0]), 1, (255, 255, 0), 1)

    for i in range(len(right_lane_idx)):
        cv2.circle(new_img, (right_lane_idx[i][1],right_lane_idx[i][0]), 1, (255, 0, 0), 1)
    
    return new_img
        


In [9]:
### Fit lane lines

def identify_lane(left_lane_idx, right_lane_idx, img_size):
    
    global prev_right_fit, right_fit_count
    
    # Obtain individual set of coordinates for each detected lane    
    left_lane_y = np.array([item[0] for item in left_lane_idx])
    left_lane_x = np.array([item[1] for item in left_lane_idx])
    right_lane_y = np.array([item[0] for item in right_lane_idx])
    right_lane_x = np.array([item[1] for item in right_lane_idx])

    # Fit a second order polynomial to lane lines
    left_fit = np.polyfit(left_lane_y , left_lane_x, 2)
    left_fit_x = left_fit[0]*left_lane_y **2 + left_fit[1]*left_lane_y  + left_fit[2]
    
    right_fit = np.polyfit(right_lane_y, right_lane_x, 2)
    right_fit_x = right_fit[0]*right_lane_y**2 + right_fit[1]*right_lane_y + right_fit[2]    

    
    # Extrapolation of left lane
    top_left_y = top_right_y = 0
    bottom_left_y = bottom_right_y = img_size[1]
    
    top_left_x = left_fit[0]*top_left_y**2 + left_fit[1]*top_left_y  + left_fit[2]
    bottom_left_x = left_fit[0]*bottom_left_y**2 + left_fit[1]*bottom_left_y  + left_fit[2]
    
    left_fit_x = np.append(np.flipud(left_fit_x), top_left_x)
    left_lane_y = np.append(np.flipud(left_lane_y), top_left_y)
    
    left_fit_x = np.append(np.flipud(left_fit_x), bottom_left_x)
    left_lane_y = np.append(np.flipud(left_lane_y), bottom_left_y)
        
    
    # Use previous frames to keep track of right lane
    if len(right_lane_x)<1000 and right_fit_count==20:
        right_fit = prev_right_fit/20
        right_fit_count = 0
        
    elif right_fit_count == 0:
        prev_right_fit = right_fit
        
    elif 0 < right_fit_count < 20:
        right_fit_count += 1
        prev_right_fit += right_fit
    
    # Extrapolation of right lane
    top_right_x = right_fit[0]*top_right_y**2 + right_fit[1]*top_right_y + right_fit[2]
    bottom_right_x = right_fit[0]*bottom_right_y**2 + right_fit[1]*bottom_right_y + right_fit[2]
    
    right_fit_x = np.append(np.flipud(right_fit_x), top_right_x)
    right_lane_y = np.append(np.flipud(right_lane_y), top_right_y)

    right_lane_y = np.append(np.flipud(right_lane_y), bottom_right_y)
    right_fit_x = np.append(np.flipud(right_fit_x), bottom_right_x)
    
    return left_lane_y, right_lane_y, left_fit_x, right_fit_x, left_fit, right_fit

In [10]:
# Draw curved lines on image
def draw_curved_line(img, line_fit):
    p = np.poly1d(line_fit)
    x = list(range(0, img.shape[0]))
    y = list(map(int, p(x)))
    points = np.array([[y1,x1] for x1, y1 in zip(x, y)])
    points = points.reshape((-1,1,2))
    
    cv2.polylines(img, np.int32([points]), False, color=(255,0,0), thickness=6)


In [11]:
# Calculate lane curvature
def lane_curvature(lane_fit_x, lane_fit_y):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meteres per pixel in x dimension

    new_fit = np.polyfit(lane_fit_y*ym_per_pix, lane_fit_x*xm_per_pix, 2)
    
    y_eval = np.max(lane_fit_y)
    
    rad_curvature = ((1 + (2*new_fit[0]*y_eval + new_fit[1])**2)**1.5)/np.absolute(2*new_fit[0])
        
    return rad_curvature

In [12]:
# Calculate offset from lane center
def distance_from_lane(img, left_lane, right_lane):
    
    xm_per_pix = 3.7/700 # meteres per pixel in x dimension
    
    img_center = (img.shape[1]//2, img.shape[0])
    
    car_pos = ((left_lane[-1] + right_lane[-1])//2 - img_center[0]) * xm_per_pix
    
    return car_pos

In [13]:
# Plot lane over original image

def draw_lane_line(image, left_lane_y, right_lane_y, left_fit_x, right_fit_x, M_inv):
    
    global frame_count
    frame_count += 1

    # Concatenate lane points
    combined_lane_left = np.array([np.flipud((np.transpose(np.vstack((left_fit_x,left_lane_y)))))])
    combined_lane_right = np.array([np.transpose(np.vstack((right_fit_x,right_lane_y)))])
    combined_lane_idx = np.hstack((combined_lane_left,combined_lane_right))
    
    # Draw lane lines and fill lane area
    img_draw = np.zeros_like(image)
    cv2.polylines(img_draw, np.int_([combined_lane_idx]), isClosed=False, color=(0,0,255), thickness = 30)
    cv2.fillPoly(img_draw, np.int_([combined_lane_idx]), (0,255, 0))
    
    # Unwarp transformed image
    new_warp = cv2.warpPerspective(img_draw, M_inv, (image.shape[1], image.shape[0]))
    new_img = cv2.addWeighted(image, 1, new_warp, 0.5, 0)
    
    # Get Radius of Curvature
    left_lane_rad = lane_curvature(left_fit_x, left_lane_y)
    right_lane_rad = lane_curvature(right_fit_x, right_lane_y)
    
    # Overlay Radius of Curvature (text)    
    cv2.putText(new_img, "Left Lane Radius: " + str("%.2f" % left_lane_rad) + " (m)", (100, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255))
    cv2.putText(new_img, "Right Lane Radius: " + str("%.2f" % right_lane_rad) + " (m)", (100, 130), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255))
    
    # Get car position
    vehicle_pos = distance_from_lane(new_img, left_fit_x, right_fit_x)
    
    # Overlay car position (text)
    cv2.putText(new_img, "Distance from car to road center: " + str("%.2f" % vehicle_pos) + " (m)", (100, 160), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255))
    
    return new_img


In [14]:
### Video pipeline
def process_video(image):
    
    # Threshold Image
    thresholded_image, M_inv = thresholded_img(image)
    img_size = [image.shape[1], image.shape[0]]
    
    # Obtain lane coordinates
    left_lane_idx, right_lane_idx = lane_coords(thresholded_image)
    
    # Identify Lane Lines
    left_lane_y, right_lane_y, left_fit_x, right_fit_x, left_fit, right_fit = identify_lane(left_lane_idx, 
                                                                                            right_lane_idx, img_size)
    
    # Draw lane lines
    final_img = draw_lane_line(image, left_lane_y, right_lane_y, left_fit_x, right_fit_x, M_inv)

    
    return final_img

In [15]:
### Prrocess video
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

right_fit_count = 0
frame_count = 0

output = 'project_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
clip = clip1.fl_image(process_video) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video project_output.mp4
[MoviePy] Writing video project_output.mp4


100%|█████████▉| 1260/1261 [04:34<00:00,  4.61it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output.mp4 

CPU times: user 9min 48s, sys: 11.3 s, total: 10min
Wall time: 4min 34s


In [ ]:
### Slider for interactive manual tuning for thresholding parameters
'''
try:
    # Undistort test image
    image = cv2.imread(images[5])
    undist_image = cv2.undistort(image, camera_mtx, camera_dist, None, camera_mtx)
    blurred_warped = cv2.GaussianBlur(warped,(5,5),0)

    def color_thresh_test(img, colorspace = cv2.COLOR_BGR2HLS, threshold_min = 140, channel = 0):

        img_convert = cv2.cvtColor(img, colorspace)
        img_ch = img_convert[:,:,channel]
        #gray_warped = cv2.cvtColor(img_ch, cv2.COLOR_RGB2GRAY)
        #binary_img = combined_binary_threshold(img_ch, sobel_kernel=7, abs_thresh=(threshold_min, 255), 
                                                            #mag_thresh=(threshold_min, 255), dir_thresh=(0.1, 0.9))
        
        thresh = (threshold_min, 255)
        
        
        binary_img = np.zeros_like(img_ch)
        ret, binary_img = cv2.threshold(img_ch, thresh[0], thresh[1], cv2.THRESH_BINARY)
        #binary_img[((img_ch > thresh[0]) & (img_ch <= thresh[1]))] = 255
        
        return binary_img

    cv2.namedWindow('image')

    def nothing(x):
        pass

    # Create slider/trackbar for threshold min/max
    cv2.createTrackbar('Threshold','image',0,255,nothing)


    # Allocate destination image
    threshold_image = np.zeros_like(warped)
    while(1):

        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break

        # Get threshold value
        threshold_val = cv2.getTrackbarPos('Threshold','image')

        threshold_image = color_thresh_test(warped, colorspace = cv2.COLOR_BGR2HLS, threshold_min = threshold_val, channel = 2)
        cv2.imshow('image', threshold_image)
    cv2.destroyAllWindows()
except:
    print("error")
    cv2.destroyAllWindows()
'''